In [1]:
import pandas as pd

In [2]:
dataset=pd.read_csv("CKD.csv")

In [3]:
dataset=pd.get_dummies(dataset,drop_first=True)

In [4]:
dataset

,age,bp,al,su,bgr,bu,sc,sod,pot,hrmo,...,pc_normal,pcc_present,ba_present,htn_yes,dm_yes,cad_yes,appet_yes,pe_yes,ane_yes,classification_yes
0,2.000000,76.459948,3.0,0.0,148.112676,57.482105,3.077356,137.528754,4.627244,12.518156,...,0,0,0,0,0,0,1,1,0,1
1,3.000000,76.459948,2.0,0.0,148.112676,22.000000,0.700000,137.528754,4.627244,10.700000,...,1,0,0,0,0,0,1,0,0,1
2,4.000000,76.459948,1.0,0.0,99.000000,23.000000,0.600000,138.000000,4.400000,12.000000,...,1,0,0,0,0,0,1,0,0,1
3,5.000000,76.459948,1.0,0.0,148.112676,16.000000,0.700000,138.000000,3.200000,8.100000,...,1,0,0,0,0,0,1,0,1,1
4,5.000000,50.000000,0.0,0.0,148.112676,25.000000,0.600000,137.528754,4.627244,11.800000,...,1,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,51.492308,70.000000,0.0,0.0,219.000000,36.000000,1.300000,139.000000,3.700000,12.500000,...,1,0,0,0,0,0,1,0,0,1
395,51.492308,70.000000,0.0,2.0,220.000000,68.000000,2.800000,137.528754,4.627244,8.700000,...,1,0,0,1,1,0,1,0,1,1
396,51.492308,70.000000,3.0,0.0,110.000000,115.000000,6.000000,134.000000,2.700000,9.100000,...,1,0,0,1,1,0,0,0,0,1
397,51.492308,90.000000,0.0,0.0,207.000000,80.000000,6.800000,142.000000,5.500000,8.500000,...,1,0,0,1,1,0,1,0,1,1


In [6]:
dataset.columns

Index(['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes', 'classification_yes'],
      dtype='object')

In [7]:
dataset["classification_yes"].value_counts()

1    249
0    150
Name: classification_yes, dtype: int64

In [9]:
indep=dataset[['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes'  ]]
dep=dataset[['classification_yes' ]]

In [10]:
indep.shape

(399, 27)

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(indep, dep, test_size = 1/3, random_state = 0)

In [13]:
from sklearn.preprocessing import StandardScaler
sc= StandardScaler()
x_train=sc.fit_transform(X_train)
x_test=sc.transform(X_test)

In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
param_grid={'solver': ['newton-cg', 'lbfgs', 'liblinear', 'saga'],
              'penalty':[ 'l2']},
grid = GridSearchCV(LogisticRegression(), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1_weighted')
grid.fit(x_train,y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


C:\Users\ELCOT\Anaconda3\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(estimator=LogisticRegression(), n_jobs=-1,
             param_grid=({'penalty': ['l2'],
                          'solver': ['newton-cg', 'lbfgs', 'liblinear',
                                     'saga']},),
             scoring='f1_weighted', verbose=3)

In [15]:
re=grid.cv_results_
grid_predictions = grid.predict(X_test)
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions)

C:\Users\ELCOT\Anaconda3\lib\site-packages\sklearn\base.py:435: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


In [16]:
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions )

In [17]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test, grid_predictions,average='weighted')
print("The f1 macro value for parameter {}:".format(grid.best_params_),f1_macro)

The f1 macro value for parameter {'penalty': 'l2', 'solver': 'newton-cg'}: 0.8628206545776206


In [18]:
print(cm)

[[39 12]
 [ 6 76]]


In [19]:
print(clf_report)

              precision    recall  f1-score   support

           0       0.87      0.76      0.81        51
           1       0.86      0.93      0.89        82

    accuracy                           0.86       133
   macro avg       0.87      0.85      0.85       133
weighted avg       0.86      0.86      0.86       133



In [20]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])

C:\Users\ELCOT\Anaconda3\lib\site-packages\sklearn\base.py:435: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


0.9526542324246772

In [21]:
table=pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_penalty,param_solver,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.326192,0.347699,0.006251,0.012501,l2,newton-cg,"{'penalty': 'l2', 'solver': 'newton-cg'}",0.981569,0.981014,0.981217,1.000000,1.000000,0.988760,0.009179,1
1,0.053249,0.006066,0.000999,0.001264,l2,lbfgs,"{'penalty': 'l2', 'solver': 'lbfgs'}",0.981569,0.981014,0.981217,1.000000,1.000000,0.988760,0.009179,1
2,0.068993,0.033950,0.006252,0.007657,l2,liblinear,"{'penalty': 'l2', 'solver': 'liblinear'}",0.963284,0.981233,0.962573,0.981217,0.981217,0.973905,0.008965,4
3,0.415284,0.390129,0.000400,0.000801,l2,saga,"{'penalty': 'l2', 'solver': 'saga'}",0.981569,0.981233,0.981217,0.981217,0.981217,0.981291,0.000139,3
